In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

DB Password: ········


In [2]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [3]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [4]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days,window_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    try:
        Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    except:
        print("DataFrame is empty. Condition does not fit to any record!")
        raise SystemExit(0)
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        #max window size
        Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) <= (window_in_days+gap_in_days)]
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    else:
        print("function not available")
    #convert list cohort_mrn_diagnosis to panda
    #condition_name= 'has_condition_'+name_feature
    #col_names = ['medical_record_number',condition_name]
    #df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    #df_final.set_index('medical_record_number', inplace=True)
    #df_final=df_mrn.merge( df_final, left_index=True, right_index=True)
    #df_final.to_parquet(name_df)
    #print(df_final)
    df_final = cohort_mrn_diagnosis
    return df_final

In [5]:
case = df_to_cohort(Case_filtered_15_540)
control = df_to_cohort(Control_filtered_15_540)

In [6]:
Med_depression_condition = (Drug('%Sertraline%') | 
                            Drug('%Bupropion%')| 
                            Drug('%Trazodone%')| 
                            Drug('%Citalopram%'))


In [ ]:
##### CASES ######

In [7]:
Med_depression_EVER = get_has_certain_condition(Med_depression_condition, Case_filtered_15_540, "Med_depression", 180 ,720, "WINDOW", "Case")

Fetching data for Drug (...)


In [8]:
Med_depression_EVER.loc[Med_depression_EVER['number_of_occurences_Med_depression'] >= 1]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_Med_depression
medical_record_number,,,,,,,,,,,,,
1126366307,334,21503.0,NaN,22557.0,21503.0,Age_BP_condition,18416,3087.0,1954,White,Christian,Male,1.0
1191350513,58,NaN,23114.0,23114.0,23114.0,Age_ICD_condition,20913,2201.0,1951,Other,Unknown,Female,4.0
1205393684,70,NaN,23159.0,23159.0,23159.0,Age_ICD_condition,11916,11243.0,1952,White,Protestant,Female,2.0
1214485155,2354,19820.0,20020.0,19838.0,19820.0,Age_BP_condition,16427,3393.0,1957,White,Catholic,Male,6.0
1232525234,258,20853.0,21840.0,20853.0,20853.0,Age_BP_condition,17997,2856.0,1954,White,Catholic,Male,1.0
1251899384,240,14321.0,12930.0,NaN,12930.0,Age_ICD_condition,11208,1722.0,1973,Unknown,Catholic,Male,1.0
1339180724,153,NaN,17274.0,17274.0,17274.0,Age_ICD_condition,15664,1610.0,1964,White,Catholic,Male,1.0
147407630,758,19970.0,NaN,19363.0,19363.0,Age_MED_condition,16839,2524.0,1957,White,Catholic,Female,1.0
1573256350,183,29383.0,29341.0,NaN,29341.0,Age_ICD_condition,26310,3031.0,1935,White,Catholic,Male,1.0


In [9]:
Med_depression_EVER.to_parquet('med_depression_window_2yr_case.parquet')

In [ ]:
##### CONTROLS######

In [24]:
Med_depression_control.to_parquet('Med_depression_control.parquet')

In [ ]:
Med_depression_EVER = get_has_certain_condition(Med_depression_condition, Control_filtered_15_540, "Med_depression", 180 ,720, "WINDOW", "Control")

Fetching data for Drug (...)


In [ ]:
Med_depression_EVER.loc[Med_depression_EVER['number_of_occurences_Med_depression'] >= 1]

In [ ]:
Med_depression_EVER.to_parquet('med_depression_window_2yr_control.parquet')